In [7]:
search_for = 85
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00439906120300293
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 69306721
type: [1, 1, 1, 1, 85] 85
cases of this type: 52200625
10000 12.626475336410563
20000 12.173368217304144
30000 40.611755840571675
40000 39.865441494022186
50000 38.633063592719296
60000 38.03484610880546
70000 38.550551008775706
80000 38.94731978429791
90000 39.310467098861714
100000 36.76910656217766
110000 35.86941484619468
120000 37.93488264796058
130000 38.34625331359269
140000 37.13037378238989
150000 39.49854871682114
160000 43.96319835998243
170000 38.2544189195241
180000 38.14108612947287
190000 38.40796348312495
200000 36.564400923770144
210000 35.305763713638264
220000 36.910190218757926
230000 37.402406528334595
240000 36.40700623937972
250000 36.31051071987236
260000 37.92666249954746
270000 36.15357010599422
280000 35.98010857907005
290000 41.029442734481435
300000 6.067482004671461
310000 6.069959936377486


3090000 36.51505210294856
3100000 17.69507046071754
3110000 11.570062671481185
3120000 11.47089261576638
3130000 15.310059656953095
3140000 37.65238385098348
3150000 34.02183468773869
3160000 33.59127600298174
3170000 32.550382257009154
3180000 31.433118457168394
3190000 31.534720143140678
3200000 32.17050676335426
3210000 32.34305247901803
3220000 34.41749211754036
3230000 32.813263269274316
3240000 34.190468169034006
3250000 34.53534018703264
3260000 33.57795282830935
3270000 31.106451191838016
3280000 32.81713213457374
3290000 31.76450611785054
3300000 32.0602773483527
3310000 34.194989738105
3320000 33.442336947013324
3330000 33.38289134443918
3340000 30.358364161964058
3350000 29.934459427035776
3360000 34.045663883874894
3370000 5.998861738913456
3380000 5.802327282166296
3390000 5.832635985053222
3400000 18.31595312308645
3410000 35.34862550425933
3420000 29.438328765792225
3430000 30.21724538741992
3440000 32.5591224861317
3450000 34.205850079534805
3460000 33.86337484860053
34

6200000 33.91940472679222
6210000 11.051455268943302
6220000 11.372047588084758
6230000 11.23486151478627
6240000 23.23536577168315
6250000 32.92324024804139
6260000 31.023433366542402
6270000 32.2426570555317
6280000 33.23227319013173
6290000 31.66135144143406
6300000 31.948548350490206
6310000 32.73706978795487
6320000 30.48138988163421
6330000 33.04479802739908
6340000 32.75014197239683
6350000 30.886528744097998
6360000 30.826881448239188
6370000 27.72960444241005
6380000 28.34802927521839
6390000 28.52297634505584
6400000 28.29222691857898
6410000 33.94983060715505
6420000 28.078617520813108
6430000 33.76185389612682
6440000 6.725449708210111
6450000 5.5190523554218744
6460000 5.5280748814504745
6470000 20.20016745541822
6480000 33.0329219996183
6490000 27.42599066508782
6500000 26.90196813066735
6510000 32.00031009910837
6520000 27.83997631985549
6530000 26.410559020895143
6540000 31.051292643452168
6550000 28.92874151995182
6560000 30.139429847759388
6570000 31.36631172430556
65

9290000 28.368730658942578
9300000 31.628183110555632
9310000 26.579564921064115
9320000 10.448754293341185
9330000 10.351910413949932
9340000 10.221835063019634
9350000 30.042021977925543
9360000 28.387098489912116
9370000 27.879371189968904
9380000 29.201821948511956
9390000 27.095004438577657
9400000 29.21661329137286
9410000 27.5582358923472
9420000 27.935001582911482
9430000 28.11797426643801
9440000 25.677482711005748
9450000 25.977888389282132
9460000 26.00497292142703
9470000 25.29087499812819
9480000 27.347530774647712
9490000 24.091901139415523
9500000 28.297616680901218
9510000 8.183910699854295
9520000 5.173752202853501
9530000 5.1368335349577725
9540000 14.177148837087021
9550000 28.073389066742656
9560000 25.30287096251368
9570000 25.572966038516075
9580000 25.89863722714124
9590000 25.85740910603063
9600000 24.544144529091803
9610000 27.67034126345569
9620000 27.354915644334064
9630000 27.64253458993144
9640000 28.75966616073197
9650000 27.169911107021758
9660000 27.4497

12290000 34.02572255895129
12300000 33.25869420661653
12310000 28.85411461503142
12320000 29.881269877920598
12330000 27.608378028081095
12340000 27.706752837937472
12350000 27.99087725172739
12360000 28.144718265908516
12370000 30.738160061327957
12380000 28.056540990052792
12390000 27.1330536228484
12400000 28.110077866135
12410000 30.387255795262888
12420000 16.34862013655591
12430000 10.225337947286366
12440000 9.96573432510696
12450000 13.176506204969188
12460000 26.07565612438277
12470000 25.090612256644228
12480000 25.333462102315146
12490000 24.202212136312188
12500000 26.40613672624559
12510000 25.890242400004084
12520000 26.589524815170385
12530000 26.792500312168325
12540000 24.32006706833413
12550000 26.251975802879215
12560000 25.553694016365615
12570000 27.35909203436877
12580000 10.184514403879225
12590000 4.9724887482712194
12600000 5.023192082145923
12610000 12.10340585930719
12620000 29.546476203546124
12630000 26.28317914363234
12640000 24.552021586742818
12650000 23

...

56990000 1.628162631423533
57000000 2.111070412537866
57010000 2.5991827308365103
57020000 1.5543661099282833
57030000 1.0728355607583457
57040000 1.7261621320769192
57050000 2.4135648149959974
57060000 1.824134906651987
57070000 1.7229473259903192
57080000 2.027613941809555
57090000 2.931090237284376
57100000 1.3452674114189942
57110000 3.2021926729169423
57120000 2.3005603185138437
57130000 2.9792742990629075
57140000 1.6789277048438918
57150000 2.5462493650099374
57160000 2.3532435870988366
57170000 1.9957601834926404
57180000 1.7718703604988852
57190000 2.365580187796692
57200000 2.2170779912897083
57210000 1.5712914570700924
57220000 2.5272401229013206
57230000 2.4898632788524298
57240000 1.9305952023237813
57250000 2.080529691249808
57260000 2.5703812390423155
57270000 2.4257983228440216
57280000 1.494042680473586
57290000 2.4307276588256688
57300000 3.137950520691693
57310000 1.3947627626541854
57320000 1.7016820479350354
57330000 2.8693134384299253
57340000 2.1273318227189777
5

59960000 0.9914317223747439
59970000 1.397575727825549
59980000 2.276352350225747
59990000 1.0104418507445256
60000000 2.051447010511239
60010000 1.8411039890756604
60020000 1.2127788660545615
60030000 0.7713592532435258
60040000 1.8259007889830867
60050000 1.992272454591592
60060000 1.4300873956663542
60070000 1.4554521912542582
60080000 1.8516015439708364
60090000 1.7303284132695396
60100000 1.1699094069877862
60110000 1.5324053089006742
60120000 1.8347666118090882
60130000 1.749915073585292
60140000 1.763357778445926
60150000 1.5556250192607841
60160000 2.3232482477511565
60170000 1.3649477789756326
60180000 2.0899673583964904
60190000 1.3911628120172024
60200000 2.3482356002423965
60210000 1.430990683065878
60220000 2.2069176829057935
60230000 1.422942861364901
60240000 1.4787267501605352
60250000 1.8737782389220792
60260000 1.882052686795493
60270000 1.834399205317047
60280000 0.969939420814991
60290000 1.8801678140606148
60300000 1.6507840153631435
60310000 1.3005886826211213
603

62890000 1.1904964161257148
62900000 1.0666752936645814
62910000 0.864392984860493
62920000 0.7662074134389957
62930000 0.7790333492411508
62940000 0.8907610784261757
62950000 0.6441181749092142
62960000 0.8696814196388257
62970000 0.717857736943543
62980000 0.7568724566502572
62990000 0.7215154716904362
63000000 0.8245303877980974
63010000 0.8595987113502224
63020000 1.1438080125982761
63030000 0.9844505233926111
63040000 1.0118466588476698
63050000 0.803921591442638
63060000 0.8770902600751586
63070000 0.6850946489117742
63080000 0.8273164925518367
63090000 0.7238955675964288
63100000 0.8024490233639876
63110000 0.987265140515705
63120000 0.9956388711856272
63130000 0.8370845777088404
63140000 1.1533031661888493
63150000 0.9870647797784672
63160000 1.1050070626978874
63170000 1.0436565694992213
63180000 1.0026366911944615
63190000 1.0839426690071026
63200000 0.6058107284806304
63210000 0.9331958070012663
63220000 0.9798107550506593
63230000 0.6001218875633744
63240000 1.1684336614540

65800000 0.22601271710276605
65810000 0.2165847721214758
65820000 0.21709709951576261
65830000 0.2347939161409537
65840000 0.23117053333891763
65850000 0.22339084058694708
65860000 0.23426688463920353
65870000 0.2033842432324158
65880000 0.2203080699770252
65890000 0.23876277862991888
65900000 0.23171327028956015
65910000 0.1958740435188214
65920000 0.21522415269190073
65930000 0.1903097582656476
65940000 0.23074115606791443
65950000 0.19227199200832845
65960000 0.19255853312516877
65970000 0.16063159184045261
65980000 0.22771914367566506
65990000 0.20834467510334648
66000000 0.2263868605088724
66010000 0.20160667595054707
66020000 0.21289934451625744
66030000 0.2067054331086477
66040000 0.20914747883194681
66050000 0.21544454892379045
66060000 0.21631426717935007
66070000 0.2095332560727994
66080000 0.21482875080100033
66090000 0.21378425808462834
66100000 0.20354965091007945
66110000 0.21066161028688485
66120000 0.19770597897934916
66130000 0.2049141174914837
66140000 0.2103545983717

68590000 0.05663951568941276
68600000 0.0491593194365965
68610000 0.030660040418790446
68620000 0.032898521086812016
68630000 0.0378385486397081
68640000 0.050525213063061236
68650000 0.050842384870111945
68660000 0.05095974915217029
68670000 0.04160896848030885
68680000 0.042949835498591264
68690000 0.04453813308515813
68700000 0.0414502522377438
68710000 0.04552728216882547
best so far: 0
type: [1, 1, 17, 5, 1] 85
cases of this type: 36125
68720000 0.04187460745886962
68730000 0.039044463509956995
68740000 0.02956574363458157
68750000 0.03815519639681445
best so far: 0
type: [1, 1, 85, 1, 1] 85
cases of this type: 7225
68760000 0.024507853741168976
best so far: 0
type: [1, 5, 1, 1, 17] 85
cases of this type: 417605
68770000 0.09086988661007087
68780000 0.06938702600662575
68790000 0.05665392469294204
68800000 0.040599832519133884
68810000 0.06441478474766678
68820000 0.06702490243557427
68830000 0.055386245833277704
68840000 0.07273247129693297
68850000 0.040622464679956435
68860000 